## Aula 2 -  PNAD C 

Nesta aula prática, vamos trabalhar com dados da PNAD Contínua trimestral, utilizando códigos em R para aprender os seguintes tópicos:


1.   Conectar a sessão ao seu google drive
2.   Leitura de microdados utilizando a biblioteca PNADcIBGE
3.   Contruir tabelas utilizando o peso amostral


📚 Hoje vamos aprender a conectar uma sessão ao seu Google drive. Então, antes de começar a trabalhar crie uma pasta no seu diretório e que irá salvar todos os arquivos que você baixar.


In [ ]:
#Conectando a sessão ao seu google drive 
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#Dessa vez, vamos trabalhar com códigos em R. Para fazer isso vamos instalar a extensão rpy2 IPython ao notebook. 
%load_ext rpy2.ipython

📌 A partir de agora, sempre que você colocar o cógido %%R no início de cada célula, você conseguirá executar comandos do R no notebook.

Utilizando a linguagem em Python, precisávamos instalar bibliotecas específicas para conseguir executar determinas comandos. No R a ideia é mesma: vamos instalar os pacotes necessários e depois temos que carrega-los toda vez que uma sessão for iniciada.

📖 Utilizaremos ao longo desta aula a biblioteca PNADcIBGE criada especificamente para leitura dos microdados da PNAD Contínua. Maiores informações sobre ela você encontra [aqui](https://cran.r-project.org/web/packages/PNADcIBGE/PNADcIBGE.pdf).

In [ ]:
#Instalando pacote da biblioteca do IBGE
%%R
install.packages("PNADcIBGE")


In [ ]:
#Carregando
%%R
library(PNADcIBGE)

In [ ]:
#Agora vamos baixar documentações e microdados da PNAD C Trimestral referentes ao 2o trimestre de 2021
#Observe que estamos baixando os microdados em uma pasta do drive (no exemplo, uma pasta chamada curso unifal)
%%R
get_pnadc(2021, quarter = 2, labels=TRUE, deflator=TRUE, savedir="/content/drive/MyDrive/unifal") 

Ao final desse procedimento em sua pasta deve conter os seguintes arquivos:
1.   Dicionario_e_input.zip
2.   PNADC_022021.zip
3.   dicionario_PNADC_microdados_trimestral.xls
4.   input_PNADC_trimestral.sas
5.   input_PNADC_trimestral.txt



In [ ]:
#Leitura dos microdados do 2o trimestre de 2021 da PNAD C:
%%R
PNAD20212 <- read_pnadc("/content/drive/MyDrive/unifal/PNADC_022021.txt", "/content/drive/MyDrive/unifal/input_PNADC_trimestral.txt", vars = NULL)

In [ ]:
#Visualizando o banco de dados
%%R
print(PNAD20212)

Variáveis que vamos trabalhar:

1.   VD4001: Condição em relação à força de trabalho na semana de referência para pessoas de 14 anos ou mais de idade
2.   VD4002: Condição de ocupação na semana de referência para pessoas de 14 anos ou mais de idade
3.   V2009: Idade do morador na data de referência
4.   V2007: Sexo
5.   V1028: Peso trimestral com correção de não entrevista com pós estratificação pela projeção de população



In [ ]:
#Criando um objeto contendo apenas essas variáveis
%%R
reduz <- c("VD4001", "VD4002", "V2009", "V2007", "V1028")
reduz_pnadc <- PNAD20212[reduz]
reduz_pnadc

# Análise exploratória dos dados

No python trabalhamos com o tipo de variáveis, e vimos que muitas vezes precisamos transformar as variáveis de numérica pra string, string de numéricas, inteiras pra fracionárias... No R faremos o mesmo, porém por meio da função str. 

In [ ]:
#Identificando o tipo das variáveis do banco de dados
%%R
str(reduz_pnadc)

In [ ]:
#Para trabalhar com as variáveis precisaremos transformar alguns de character(string)em numéricas
%%R
reduz_pnadc$VD4001=as.numeric(reduz_pnadc$VD4001)
reduz_pnadc$VD4002=as.numeric(reduz_pnadc$VD4002)
reduz_pnadc$V2007=as.numeric(reduz_pnadc$V2007)
reduz_pnadc$VD4009=as.numeric(reduz_pnadc$VD4009)



In [ ]:
#Instalando o pacote que permite utilizar o peso amostral
%%R
install.packages("expss")


In [ ]:
%%R
library(expss)

In [ ]:
#Variável condição de atividade (VD4001)
%%R
table(reduz_pnadc$VD4001)


In [ ]:
#Adicionando os rótulos criando uma nova variável (condat)
%%R
reduz_pnadc$condat<-factor(reduz_pnadc$VD4001,labels=c('PEA', 'Inativos'))

In [ ]:
#Gerando a frequência ponderada
%%R
fre(reduz_pnadc$condat, weight = reduz_pnadc$V1028)

In [ ]:
#Se eu quiser exportar a tabela que geramos para o excel (ela ficará salva na sua sessão)
%%R
tab1=fre(reduz_pnadc$condat, weight = reduz_pnadc$V1028)
write.table(tab1,file="questao1.xls", sep = "\t", dec = ",")

In [ ]:
#Variável condição de ocupação (VD4002)
%%R
table(reduz_pnadc$VD4002)

In [ ]:
#Adicionando os rótulos criando uma nova variável (condocup)
%%R
reduz_pnadc$condocup<-factor(reduz_pnadc$VD4002,labels=c('Ocupadas', 'Desocupadas'))


In [ ]:
#Gerando a frequência ponderada
%%R
fre(reduz_pnadc$condocup, weight = reduz_pnadc$V1028)

In [ ]:
#Exportanto a tabela que geramos para o excel (ela ficará salva na sua sessão)
%%R
tab2=fre(reduz_pnadc$condocup, weight = reduz_pnadc$V1028)
write.table(tab2,file="questao2.xls", sep = "\t", dec = ",")

In [ ]:
#Variável idade (V2009):
%%R
summary(reduz_pnadc$V2009)

In [ ]:
#Construindo faixas etárias (IDADECAT)
%%R
reduz_pnadc$IDADECAT <- factor(cut(reduz_pnadc$V2009, breaks=c(14,20,25,30,35,40,45,50,55,60,65,112), labels=c("14-19","20-24","25-29","30-34","35-39","40-44","45-49", "50-54","55-59", "60-64","65+"),right=FALSE))

In [ ]:
#Gerando a frequência ponderada
%%R
fre(reduz_pnadc$IDADECAT, weight = reduz_pnadc$V1028)

In [ ]:
#Exportanto a tabela que geramos para o excel (ela ficará salva na sua sessão)
%%R
tab3=fre(reduz_pnadc$IDADECAT, weight = reduz_pnadc$V1028)
write.table(tab3,file="questao3.xls", sep = "\t", dec = ",")

In [ ]:
#Variável sexo (V2007)
%%R
table(reduz_pnadc$V2007)

In [ ]:
#Adicionando os rótulos criando uma nova variável (sexo)
%%R
reduz_pnadc$sexo<-factor(reduz_pnadc$V2007,labels=c('Homens', 'Mulheres'))

In [ ]:
#Gerando a frequência ponderada
%%R
fre(reduz_pnadc$sexo, weight = reduz_pnadc$V1028)

In [ ]:
#Salvando a base de dados em seu drive
%%R
save(reduz_pnadc,file="/content/drive/MyDrive/unifal/reduz_pnadc.Rdata")

In [ ]:
#Abrindo a base
%%R
load("/content/drive/MyDrive/unifal/reduz_pnadc.Rdata")